<a href="https://colab.research.google.com/github/marcosfelt/latex2speech/blob/main/tts_latex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to speech for Latex

This notebook converts Latex into speech. It's useful for having your papers read back to you during editing/proofreading.

How to use:


1. Execute the cells under "Setup" to install all the necessary packages
2. Paste your latex code into the "texts" list and execute the cells.
3. Execte the remaining cells and you'll get audio players at the end for each paragraph.

## Setup

In [ ]:
# Needed for inflect
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Install packages
!pip install TTS inflect

In [122]:
from TTS.api import TTS
import re
import textwrap
import inflect
import IPython
from IPython.display import display, clear_output

# Conversion of numbers
p = inflect.engine()
def convert_numbers(matchobj):
    return p.number_to_words(matchobj.group(0))

## Generate speech

In [116]:
# Paste the Latex text here. Each element in the list is a paragraph
texts = [
    "In the fine chemicals industry, reaction screening and optimisation are essential to development of new products. However, this screening can be extremely time and labor intensive, especially when intuition is used. Machine learning offers a solution through iterative suggestions of new experiments based on past experimental data, but knowing which machine learning strategy to apply in a particular case is still difficult. Here, we develop chemically-motivated virtual benchmarks for reaction optimisation and compare several strategies on these benchmarks. The benchmarks and strategies are encompassed in an open-source framework named Summit. The results of our tests show that Bayesian optimisation strategies perform very well across the types of problems faced in chemical reaction optimisation, while many strategies commonly used in reaction optimisation fail to find optimal solutions."
]

In [125]:
# Clean up latex
for i in range(len(texts)):
  # Strip latex citations and references
  texts[i] = re.sub(r"\\cite\{[A-za-z\d,\s]+\}", "", texts[i])
  texts[i] = re.sub(r"\\ref\{[A-za-z\d,\s\-\_:]+\}", "", texts[i])
  # Convert numbers to words
  texts[i] = re.sub(r"\d+(\.\d+)?", convert_numbers, texts[i])
  # Remove random latex symbols
  for s in ["$", "\\", "{" ,"}"]:
    texts[i] = texts[i].replace(s, "")
  texts[i] = texts[i].replace("_", "-")


model_name = TTS.list_models()[0]
wavs = []
for text in texts:
  tts = TTS(model_name, gpu=True, progress_bar=False)
  wav = tts.tts(text, speaker=tts.speakers[1], language=tts.languages[0])
  wavs.append(wav)
clear_output(wait=True)

for text, wav in zip(texts, wavs):
  print(" \n".join(textwrap.wrap(text, width=70)))
  print()
  display(IPython.display.Audio(wav, rate=16000))

Displaying audio files
In the fine chemicals industry, reaction screening and optimisation 
are essential to development of new products. However, this screening 
can be extremely time and labor intensive, especially when intuition 
is used. Machine learning offers a solution through iterative 
suggestions of new experiments based on past experimental data, but 
knowing which machine learning strategy to apply in a particular case 
is still difficult. Here, we develop chemically-motivated virtual 
benchmarks for reaction optimisation and compare several strategies on 
these benchmarks. The benchmarks and strategies are encompassed in an 
open-source framework named Summit. The results of our tests show that 
Bayesian optimisation strategies perform very well across the types of 
problems faced in chemical reaction optimisation, while many 
strategies commonly used in reaction optimisation fail to find optimal 
solutions.

